In [3]:
pip install pandas==1.3.4

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install fcsparser
pip install pandas
pip install scanpy

In [3]:
!pip install tensorflow
import scaespy.scaespy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 511.7 MB 15 kB/s s eta 0:00:01     |██████▍                         | 101.9 MB 59.8 MB/s eta 0:00:07     |█████████                       | 143.1 MB 31.4 MB/s eta 0:00:12     |█████████                       | 144.3 MB 8.0 MB/s eta 0:00:46     |█████████▏                      | 147.4 MB 8.0 MB/s eta 0:00:46     |█████████▎                      | 148.5 MB 8.0 MB/s eta 0:00:46     |█████████▍                      | 150.6 MB 8.0 MB/s eta 0:00:45     |██████████                      | 160.7 MB 8.0 MB/s eta 0:00:44     |██████████▎                     | 164.0 MB 24.2 MB/s eta 0:00:15     |██████████▍                     | 165.8 MB 24.2 MB/s eta 0:00:15     |██████████▍                     | 166.6 MB 24.2 MB/s eta 0:00:15     |██████████▋                     | 169.2 MB 24.2 MB/s eta 0:00:15     |██████████▋                     | 170.1 MB 24.2 MB/s eta 0:00:15     |██████████▉

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [2]:
import pandas as pd
import scanpy as sc
import os

In [10]:
# IF this doesn't work, try reading fcs natively using python
data_dir = f'/home/razgaon/ondemand/data/sys/myjobs/projects/default/3/'
NSCLC_pickle = f'{data_dir}NSCLC_prepared_data_test.pickle'
PBMC_pickle = f'{data_dir}PBMC_prepared_data_test.pickle'
NSCLC_df = pd.read_pickle(NSCLC_pickle)
PBMC_df = pd.read_pickle(PBMC_pickle)


# Try to add an id that maps to the fcs file name


In [11]:
PBMC_df.head()

,id,CD11b,Beads,CD80,FoxP3,CD33,CXCR5,CCR6,Gata3,RORgT,...,EOMES,CD25,HuCD45,Blimp1,CD34,Tbet,CD4,CD127,sample,batch
0,1,0.000000,0.198690,0.19869,1.137982,0.732668,0.881374,0.390035,2.187422,0.881374,...,0.568825,0.732668,3.719027,0.198690,0.00000,1.137982,3.709282,1.248983,1.0,B
1,2,0.732668,0.000000,0.19869,1.609438,1.015973,0.000000,0.732668,0.881374,0.390035,...,0.198690,2.915291,3.888150,0.568825,0.19869,1.137982,4.107860,2.524138,1.0,B
2,3,0.198690,0.000000,0.00000,0.881374,0.198690,0.000000,0.390035,2.672016,0.000000,...,0.732668,0.390035,4.039846,0.881374,0.00000,0.732668,0.881374,3.146005,1.0,B
3,4,0.000000,0.732668,0.00000,0.198690,0.390035,0.390035,0.390035,1.248983,0.881374,...,0.000000,0.000000,3.617031,1.015973,0.19869,2.555494,0.568825,0.732668,1.0,B
4,5,1.443635,0.000000,0.00000,0.732668,0.000000,3.017929,2.957612,0.881374,0.568825,...,0.000000,1.137982,3.981897,0.390035,0.00000,0.198690,1.350441,0.198690,1.0,B


In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

NSCLC_markers = [col for col in NSCLC_df.columns if col not in ['id', 'Time', 'Eventlength', 'sample', 'batch']]
NSCLC_df[NSCLC_markers] = scaler.fit_transform(NSCLC_df[NSCLC_markers])

PBMC_markers = [col for col in PBMC_df.columns if col not in ['id', 'sample', 'batch']]
PBMC_df[PBMC_markers] = scaler.fit_transform(PBMC_df[PBMC_markers])




In [20]:
df = pd.concat([PBMC_df, NSCLC_df])
# Confirm that concat is creating new columns

In [23]:
# Data loader
df.head()

# Convert metals to their marker nadmes to ensure consistency when merging dataframes


,batch
0,B
1,B
2,B
3,B
4,B
...,...
261405,A
261406,A
261407,A
261408,A


In [30]:
class VAE(nn.Module):
    def _init_(self, x_dim, latent_dim=100, hdim=128):
        super(VAE, self)._init_()
        self.x_dim = x_dim
        self.latent_dim = latent_dim
        self.fc1 = nn.Linear(x_dim , hdim)
        self.fc21 = nn.Linear(hdim, latent_dim)
        self.fc22 = nn.Linear(hdim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hdim)
        self.fc4 = nn.Linear(hdim, x_dim)
    def encode(self, x):
        x_ = torch.cat([x], dim=-1)
        h1 = F.relu(self.fc1(x_))
        return self.fc21(h1), self.fc22(h1)
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)
    def reparameterize(self, mu, var):
        if self.distance == 'standard':
            if self.training:
                std = var.mul(0.5).exp_()
                eps = Variable(std.data.new(std.size()).normal_())
                result = eps.mul(std).add_(mu)
                return result
            else:
                return mu
    def forward(self, x, c):
        mu, logvar = self.encode(x.view(-1, self.x_dim))
        latent = self.reparameterize(mu, logvar)
        reconstructed = self.decode(latent)
        return reconstructed, latent, dist


def compute_kernel(x, y, device):
    x_size = x.to(device).size(0)
    y_size = y.to(device).size(0)
    dim = x.size(1)
    x = x.unsqueeze(1)
    y = y.unsqueeze(0)
    tiled_x = x.expand(x_size, y_size, dim).to(device)
    tiled_y = y.expand(x_size, y_size, dim).to(device)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input)
def compute_mmd(x, y, device):
    x_kernel = compute_kernel(x, x, device)
    y_kernel = compute_kernel(y, y, device)
    xy_kernel = compute_kernel(x, y, device)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd
def vae_loss_function(inputs, encoded, decoded, true_samples, device):
    #reconstruction loss - decoding
    nll = (decoded - inputs).pow(2).mean()
    #distance + reconstruction loss
    decoder_loss = compute_mmd(true_samples, encoded.to(device), device) + nll
    return(decoder_loss)

In [31]:
# Demonstration of merging dataframes using concatenation
df1 = pd.DataFrame([[1,2,3,4,5],[6,7,8,9,10]], columns = ["A", "B", "C", "D", "E"])
df2 = pd.DataFrame([[1,2,3,4,5],[6,7,8,9,10]], columns = ["F", "G", "H", "D", "E"])
x = pd.concat([df1, df2])
x


,A,B,C,D,E,F,G,H
0,1.0,2.0,3.0,4,5,NaN,NaN,NaN
1,6.0,7.0,8.0,9,10,NaN,NaN,NaN
0,NaN,NaN,NaN,4,5,1.0,2.0,3.0
1,NaN,NaN,NaN,9,10,6.0,7.0,8.0
